#Tradução Automática

## Aluno: Yago Phellipe Matos Lopes (A2022.2A.0153:)
### Curso: Ciência da Computação

In [14]:
import os
import hashlib
import urllib.request
import zipfile
import re

In [15]:
class DataModule:
    """Classe base que simula d2l.DataModule"""
    def __init__(self, root='../data'):
        self.root = root
        os.makedirs(self.root, exist_ok=True)

In [16]:




class TraducaoDownload(DataModule):
    def _download(self):
        """Download e extração do conjunto de dados francês-inglês"""
        # URL e hash do arquivo
        url = 'https://d2l-data.s3-accelerate.amazonaws.com/fra-eng.zip'
        sha1_hash = '94646ad1522d915e7b0f9296181140edcf86a4f5'

        # Cria pasta data se não existir
        os.makedirs(self.root, exist_ok=True)

        # Nome do arquivo
        fname = os.path.join(self.root, url.split('/')[-1])

        # Verifica se o arquivo já existe e tem o hash correto
        if os.path.exists(fname):
            sha1 = hashlib.sha1()
            with open(fname, 'rb') as f:
                while True:
                    data = f.read(1048576)
                    if not data:
                        break
                    sha1.update(data)
            if sha1.hexdigest() == sha1_hash:
                print(f"Arquivo {fname} já baixado e verificado")
            else:
                print(f"Hash do arquivo {fname} incorreto, baixando novamente")
                self._download_file(url, fname)
        else:
            print(f"Baixando {fname} de {url}...")
            self._download_file(url, fname)

        # Extrai arquivo
        base_dir = os.path.dirname(fname)
        data_dir = os.path.join(base_dir, 'fra-eng')
        if not os.path.exists(data_dir):
            print(f"Extraindo {fname}...")
            with zipfile.ZipFile(fname, 'r') as zf:
                zf.extractall(base_dir)

        # Lê o arquivo extraído
        with open(os.path.join(data_dir, 'fra.txt'), encoding='utf-8') as f:
            return f.read()

    def _download_file(self, url, fname):
        """Baixa o arquivo com progresso"""
        try:
            urllib.request.urlretrieve(url, fname)
        except:
            # Fallback para dados embutidos se download falhar
            print("Erro ao baixar. Criando um arquivo de exemplo.")
            fallback_data = (
                "Go.\tVa !\n"
                "Hi.\tSalut !\n"
                "Run!\tCours !\n"
                "Run!\tCourez !\n"
                "Who?\tQui ?\n"
                "Wow!\tÇa alors !\n"
            )
            os.makedirs(os.path.dirname(fname), exist_ok=True)
            with open(fname, 'w', encoding='utf-8') as f:
                f.write(fallback_data)

            # Cria diretório para extração
            extract_dir = os.path.join(os.path.dirname(fname), 'fra-eng')
            os.makedirs(extract_dir, exist_ok=True)

            # Cria arquivo fra.txt
            with open(os.path.join(extract_dir, 'fra.txt'), 'w', encoding='utf-8') as f:
                f.write(fallback_data)

In [17]:

def preprocess_nmt(text):
    """Preprocessa o conjunto de dados de tradução."""
    def no_space(char, prev_char):
        return char in set(',.!?') and prev_char != ' '

    # Substituir espaços especiais e converter para minúsculas
    text = text.replace('\u202f', ' ').replace('\xa0', ' ').lower()

    # Inserir espaço entre palavras e pontuação
    out = [' ' + char if i > 0 and no_space(char, text[i - 1]) else char
           for i, char in enumerate(text)]

    return ''.join(out)

In [18]:
def tokenize(text):
    """Tokenização simples por espaço."""
    return text.split()

In [19]:
def build_vocab(data, min_freq=2):
    """Construção do vocabulário."""
    counter = {}
    for sentence in data:
        for token in tokenize(sentence):
            if token not in counter:
                counter[token] = 0
            counter[token] += 1

    # Tokens especiais
    specials = ["<unk>", "<pad>", "<bos>", "<eos>"]

    # Cria vocabulário com tokens que aparecem pelo menos min_freq vezes
    vocab = {
        token: idx
        for idx, token in enumerate(
            specials +
            [token for token, count in counter.items() if count >= min_freq]
        )
    }

    # Dicionário reverso (para lookup rápido)
    inv_vocab = {idx: token for token, idx in vocab.items()}

    return vocab, inv_vocab, counter

In [20]:
def process_dataset(raw_text, num_examples=None):
    """Processa o texto bruto em pares de sentenças."""
    text = preprocess_nmt(raw_text)
    lines = text.split('\n')
    pairs = []

    for line in lines:
        if not line.strip():
            continue
        parts = line.split('\t')
        if len(parts) >= 2:
            pairs.append([parts[0].strip(), parts[1].strip()])

    if num_examples is not None:
        pairs = pairs[:num_examples]

    return pairs

In [21]:
def main():
    # Download e leitura dos dados
    data = TraducaoDownload()
    try:
        raw_text = data._download()
    except Exception as e:
        print(f"Erro ao baixar: {e}")
        return

    print("Amostra do texto bruto:")
    print(raw_text[:75])

    # Pré-processar e dividir em pares
    text = preprocess_nmt(raw_text)
    print("\nAmostra do texto pré-processado:")
    print(text[:80])

    # Processar o conjunto de dados com diferentes tamanhos
    for num_examples in [1000, 5000, 10000]:
        print(f"\n--- Análise com {num_examples} exemplos ---")

        # Processar pares de frases
        pairs = process_dataset(raw_text, num_examples)

        # Separar fonte e alvo
        src_sentences = [src for src, _ in pairs]
        tgt_sentences = [tgt for _, tgt in pairs]

        # Construir vocabulários
        vocab_src, inv_vocab_src, counter_src = build_vocab(src_sentences)
        vocab_tgt, inv_vocab_tgt, counter_tgt = build_vocab(tgt_sentences)

        # Imprimir estatísticas
        print(f"Total de pares: {len(pairs)}")
        print(f"Tamanho do vocabulário (Fonte): {len(vocab_src)}")
        print(f"Tamanho do vocabulário (Alvo): {len(vocab_tgt)}")

        # Imprimir algumas amostras
        print("\nAmostras de pares:")
        for i in range(min(3, len(pairs))):
            print(f"Fonte: {pairs[i][0]}")
            print(f"Alvo: {pairs[i][1]}")

if __name__ == "__main__":
    main()

Arquivo ../data/fra-eng.zip já baixado e verificado
Amostra do texto bruto:
Go.	Va !
Hi.	Salut !
Run!	Cours !
Run!	Courez !
Who?	Qui ?
Wow!	Ça alors !


Amostra do texto pré-processado:
go .	va !
hi .	salut !
run !	cours !
run !	courez !
who ?	qui ?
wow !	ça alors !

--- Análise com 1000 exemplos ---
Total de pares: 1000
Tamanho do vocabulário (Fonte): 266
Tamanho do vocabulário (Alvo): 321

Amostras de pares:
Fonte: go .
Alvo: va !
Fonte: hi .
Alvo: salut !
Fonte: run !
Alvo: cours !

--- Análise com 5000 exemplos ---
Total de pares: 5000
Tamanho do vocabulário (Fonte): 875
Tamanho do vocabulário (Alvo): 1230

Amostras de pares:
Fonte: go .
Alvo: va !
Fonte: hi .
Alvo: salut !
Fonte: run !
Alvo: cours !

--- Análise com 10000 exemplos ---
Total de pares: 10000
Tamanho do vocabulário (Fonte): 1505
Tamanho do vocabulário (Alvo): 2250

Amostras de pares:
Fonte: go .
Alvo: va !
Fonte: hi .
Alvo: salut !
Fonte: run !
Alvo: cours !
